In [1]:
import pandas as pd
import numpy as np
import time
import os
import zipfile
import pandas_gbq
import glob
from google.cloud import bigquery
import re

In [2]:
client = bigquery.Client()

In [24]:
query = """ SELECT ANO as ano, MES as mes, TIPO_PENSAO as tipo_pensao, DATA_INICIO_PENSAO as data_inicio_pensao, 
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2024_abr_militares_pensionistas_cadastro` 
        """  

In [25]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')
df

Downloading: 100%|██████████|


,ano,mes,tipo_pensao,data_inicio_pensao,total
0,2024,4,Não informado,04/01/2024,1
1,2024,4,Cônjuge / Viúva (o),21/04/1979,1
2,2024,4,Cônjuge / Viúva (o),04/12/1979,1
3,2024,4,Cônjuge / Viúva (o),23/05/1981,1
4,2024,4,Cônjuge / Viúva (o),22/09/1967,1
...,...,...,...,...,...
237588,2024,4,Outros (Pessoas sem Vínculo Militar),01/07/2008,1
237589,2024,4,Filho(a) adotivo ou Enteado(a),16/03/2017,1
237590,2024,4,Filho(a) adotivo ou Enteado(a),18/06/2019,1
237591,2024,4,Filho(a) adotivo ou Enteado(a),09/06/2023,1


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237593 entries, 0 to 237592
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   ano                 237593 non-null  Int64 
 1   mes                 237593 non-null  Int64 
 2   tipo_pensao         237593 non-null  object
 3   data_inicio_pensao  237551 non-null  object
 4   total               237593 non-null  Int64 
dtypes: Int64(3), object(2)
memory usage: 9.7+ MB


In [26]:
pensoes = df['data_inicio_pensao'].isnull()
pensoes.value_counts()

False    237551
True         42
Name: data_inicio_pensao, dtype: int64

In [31]:
df = df.dropna() # Foram dropadas 42 linhas, equivalente ao total de valores nulos indicados acima
df

,ano,mes,tipo_pensao,data_inicio_pensao,total
0,2024,4,Não informado,04/01/2024,1
1,2024,4,Cônjuge / Viúva (o),21/04/1979,1
2,2024,4,Cônjuge / Viúva (o),04/12/1979,1
3,2024,4,Cônjuge / Viúva (o),23/05/1981,1
4,2024,4,Cônjuge / Viúva (o),22/09/1967,1
...,...,...,...,...,...
237588,2024,4,Outros (Pessoas sem Vínculo Militar),01/07/2008,1
237589,2024,4,Filho(a) adotivo ou Enteado(a),16/03/2017,1
237590,2024,4,Filho(a) adotivo ou Enteado(a),18/06/2019,1
237591,2024,4,Filho(a) adotivo ou Enteado(a),09/06/2023,1


In [34]:
df['data_inicio_pensao'] = pd.to_datetime(df['data_inicio_pensao'], dayfirst=True, format= "%d/%m/%Y") #transformando em data

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_17600\3566975300.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['data_inicio_pensao'] = pd.to_datetime(df['data_inicio_pensao'], dayfirst=True, format= "%d/%m/%Y") #transformando em data


In [36]:
df['ano_inicio_pensao'] = df['data_inicio_pensao'].dt.year #pegando apenas o ano

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_17600\3811510978.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ano_inicio_pensao'] = df['data_inicio_pensao'].dt.year #pegando apenas o ano


In [40]:
df['ano_inicio_pensao'].unique()

array([2024, 1979, 1981, 1967, 2005, 1976, 1972, 2003, 2017, 2016, 1973,
       1993, 1980, 1978, 2010, 1959, 2014, 1977, 1961, 1963, 2018, 2015,
       2002, 2023, 1970, 1974, 1975, 1966, 2022, 1956, 2013, 1971, 1962,
       2019, 2009, 1982, 2020, 1969, 1958, 1965, 1964, 2011, 2000, 1983,
       2021, 1968, 2012, 2001, 1951, 2004, 2008, 1991, 1960, 1948, 1953,
       1985, 1984, 2006, 1950, 1995, 2007, 1957, 1944, 1955, 1943, 1999,
       1952, 1945, 1949, 1954, 1998, 1994, 1933, 1996, 1941, 1937, 1986,
       1932, 1987, 1988, 1990, 1989, 1997, 1947, 1992, 1946, 1940, 1942,
       1936, 1939, 1934, 1930, 1938], dtype=int64)

In [41]:
limites = [0, 1939, 1949, 1959, 1969, 1979, 1989, 1999, 2009, 2019, 2024]
categorias = [1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020]

df['decada_inicio_pensao'] = pd.cut(df['ano_inicio_pensao'], bins=limites, labels=categorias)


C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_17600\3459730341.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['decada_inicio_pensao'] = pd.cut(df['ano_inicio_pensao'], bins=limites, labels=categorias)


In [44]:
df['tipo_pensao'].unique()

array(['Não informado', 'Cônjuge / Viúva (o)', 'Companheiro (a)',
       'Pessoa desquitada, separada judicialmente, divorciada do instituidor ou ex-convivente',
       'Filha', 'Filho', 'Mãe', 'Pai', 'Neto (a)', 'Irmão (ã)',
       'Menor sob guarda ou tutela',
       'Pessoa designada (Beneficiário instituído)',
       'Ex-combatente (o próprio)',
       'Outros (Pessoas sem Vínculo Militar)',
       'Filho(a) adotivo ou Enteado(a)'], dtype=object)

In [45]:
df

,ano,mes,tipo_pensao,data_inicio_pensao,total,ano_inicio_pensao,decada_inicio_pensao
0,2024,4,Não informado,2024-01-04,1,2024,2020
1,2024,4,Cônjuge / Viúva (o),1979-04-21,1,1979,1970
2,2024,4,Cônjuge / Viúva (o),1979-12-04,1,1979,1970
3,2024,4,Cônjuge / Viúva (o),1981-05-23,1,1981,1980
4,2024,4,Cônjuge / Viúva (o),1967-09-22,1,1967,1960
...,...,...,...,...,...,...,...
237588,2024,4,Outros (Pessoas sem Vínculo Militar),2008-07-01,1,2008,2000
237589,2024,4,Filho(a) adotivo ou Enteado(a),2017-03-16,1,2017,2010
237590,2024,4,Filho(a) adotivo ou Enteado(a),2019-06-18,1,2019,2010
237591,2024,4,Filho(a) adotivo ou Enteado(a),2023-06-09,1,2023,2020


In [63]:
df['decada_inicio_pensao'] = df['decada_inicio_pensao'].astype(int)

In [54]:
pensoes = {
    "Filho" : "Filhos, netos, enteados ou menores em tutela",
    "Filha" : "Filhos, netos, enteados ou menores em tutela",
    "Menor sob guarda ou tutela" : "Filhos, netos, enteados ou menores em tutela",
    "Neto (a)" : "Filhos, netos, enteados ou menores em tutela",
    "Filho(a) adotivo ou Enteado(a)" : "Filhos, netos, enteados ou menores em tutela",
    "Cônjuge / Viúva (o)" : "Cônjuge ou ex-cônjuge",
    "Companheiro (a)" : "Cônjuge ou ex-cônjuge",
    "Pessoa desquitada, separada judicialmente, divorciada do instituidor ou ex-convivente" : "Cônjuge ou ex-cônjuge",
    "Mãe":"Pais",
    "Pai":"Pais",
    "Irmão (ã)":"Irmão (ã)",
    "Ex-combatente (o próprio)":"Ex-combatente (o próprio)",
    "Pessoa designada (Beneficiário instituído)":"Pessoa designada (Beneficiário instituído)",
    "Não informado" : "Não informado",
    "Outros (Pessoas sem Vínculo Militar)" : "Outros (Pessoas sem Vínculo Militar)"
    }


In [55]:
def categorizando(x):
    if x in pensoes:
        return pensoes[x]


In [56]:
df['tipo_pensao_agrupado'] = df['tipo_pensao'].apply(categorizando)

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_17600\1944507786.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tipo_pensao_agrupado'] = df['tipo_pensao'].apply(categorizando)


## Tratamento final

In [58]:
df= df[['ano', 'mes', 'data_inicio_pensao', 'ano_inicio_pensao', 'decada_inicio_pensao', 'tipo_pensao','tipo_pensao_agrupado', 'total']]
df

,ano,mes,data_inicio_pensao,ano_inicio_pensao,decada_inicio_pensao,tipo_pensao,tipo_pensao_agrupado,total
0,2024,4,2024-01-04,2024,2020,Não informado,Não informado,1
1,2024,4,1979-04-21,1979,1970,Cônjuge / Viúva (o),Cônjuge ou ex-cônjuge,1
2,2024,4,1979-12-04,1979,1970,Cônjuge / Viúva (o),Cônjuge ou ex-cônjuge,1
3,2024,4,1981-05-23,1981,1980,Cônjuge / Viúva (o),Cônjuge ou ex-cônjuge,1
4,2024,4,1967-09-22,1967,1960,Cônjuge / Viúva (o),Cônjuge ou ex-cônjuge,1
...,...,...,...,...,...,...,...,...
237588,2024,4,2008-07-01,2008,2000,Outros (Pessoas sem Vínculo Militar),Outros (Pessoas sem Vínculo Militar),1
237589,2024,4,2017-03-16,2017,2010,Filho(a) adotivo ou Enteado(a),"Filhos, netos, enteados ou menores em tutela",1
237590,2024,4,2019-06-18,2019,2010,Filho(a) adotivo ou Enteado(a),"Filhos, netos, enteados ou menores em tutela",1
237591,2024,4,2023-06-09,2023,2020,Filho(a) adotivo ou Enteado(a),"Filhos, netos, enteados ou menores em tutela",1


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 237551 entries, 0 to 237592
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   ano                   237551 non-null  Int64         
 1   mes                   237551 non-null  Int64         
 2   data_inicio_pensao    237551 non-null  datetime64[ns]
 3   ano_inicio_pensao     237551 non-null  int64         
 4   decada_inicio_pensao  237551 non-null  int32         
 5   tipo_pensao           237551 non-null  object        
 6   tipo_pensao_agrupado  237551 non-null  object        
 7   total                 237551 non-null  Int64         
dtypes: Int64(3), datetime64[ns](1), int32(1), int64(1), object(2)
memory usage: 16.1+ MB


In [61]:
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')

In [66]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
        bigquery.SchemaField('mes','INTEGER',description='Mês de referência da observação'), 
        bigquery.SchemaField('data_inicio_pensao','DATETIME',description='Data de início da pensão'),
        bigquery.SchemaField('ano_inicio_pensao','INTEGER',description='Ano de início da pensão extraido da data'),
        bigquery.SchemaField('decada_inicio_pensao','INTEGER',description='Década de início da pensão'),
        bigquery.SchemaField('tipo_pensao','STRING',description='Tipo de pensão'),
        bigquery.SchemaField('tipo_pensao_agrupado','STRING',description='Categorização das pensões'),
        bigquery.SchemaField('total','INTEGER',description='Quantidade total da observação')
        ]

In [67]:
table_ref = dataset_ref.table('MILITARES_inicio_pensoes_total')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 


LoadJob<project=repositoriodedadosgpsp, location=US, id=1f1a331d-c431-4419-98c5-4b3a63e09e36>